use this notebook to see today's bet at the beginning of a day

In [28]:
import pandas as pd, numpy as np
import statsapi
from statsapi import player_stat_data
import requests
from datetime import datetime, timedelta
import numpy as np
import math, os
import meteostat
import pprint
import pickle
import copy

import pycaret
from pycaret import classification
import model.common, model.odds_eval
import odds_data.query_bq_odds
import importlib

In [17]:
from static_data.load_static_data import *

In [79]:
importlib.reload(model.common)
importlib.reload(model.odds_eval)
importlib.reload(odds_data.query_bq_odds)

<module 'odds_data.query_bq_odds' from '/home/junlim/projects/mlb-props/odds_data/query_bq_odds.py'>

In [53]:
collect_data_Base_dir = 'collect_data'
df_game_matchup_total = pd.read_pickle(f'{collect_data_Base_dir}/df_game_matchup_total.pkl')
df_game_matchup_2023 = pd.read_pickle(f'{collect_data_Base_dir}/df_game_matchup_2023.pkl')
print(f'df_game_matchup_total: {len(df_game_matchup_total)}, df_game_matchup_2023: {len(df_game_matchup_2023)}')

df_game_matchup_total: 326057, df_game_matchup_2023: 16412


In [97]:
import os
if os.path.exists('credential.json'):
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.join(os.getcwd(), 'credential.json')
property = 'all'
odds_data.query_bq_odds.download_property(property = property)
df_odds_2023 = pd.read_pickle(f'odds_data/df_odds_{property}.pkl')

running querry

    SELECT * 
    FROM `trading-290017.major_league_baseball.odds_batter_prop`
    WHERE TRUE
    AND game_date >= "2023-04-01"



# 2023 season prediction - profitable models

In [209]:
feature_columns_list = [model.common.features_1hits_recorded, model.common.features_1hstrikeouts_recorded]
target_column_list = [model.common.target_1hits_recorded, model.common.target_1hstrikeouts_recorded]
model_file_name_list = [model.common.model_1hits_file_name, model.common.model_1hstrikeouts_file_name]

In [210]:
for feature_columns, target_column, model_file_name in zip(feature_columns_list, target_column_list, model_file_name_list):    
    print(target_column, model_file_name)
    regression_model = pycaret.classification.load_model(model_file_name)
    df_prediction_2023 = model.odds_eval.df_prediction_add_odd(df_game_matchup_2023[['game_id'] + feature_columns + [target_column]], regression_model)
    df_prediction_odds_2023 = model.odds_eval.merge_df_prediction_over_odds(df_prediction_2023, df_odds_since_20230529[df_odds_2023.property == 'Hits'], target_column, 1.0)
    df_confident_prediction_odds_2023 = model.odds_eval.get_df_confident_prediction_odds(df_prediction_odds_2023, target_column, score_threshold = 0.75)
    df_advantageous_prediction_odds_2023 = model.odds_eval.get_df_advantageous_prediction_odds(df_prediction_odds_2023, target_column, prediction_diff_threshold = 0.05, score_threshold = 0.70)

batting_1hits_recorded model/batter_1hits_regression_model
Transformation Pipeline and Model Successfully Loaded
hit recorded ratio: 0.75 (51 out of 68)
profit: 5.215812654719599
hit recorded ratio: 0.8571428571428571 (6 out of 7)
profit: 1.689879297022154
batting_1strikeOuts_recorded model/batter_1hstrikeouts_regression_model
Transformation Pipeline and Model Successfully Loaded
hit recorded ratio: 0.8333333333333334 (135 out of 162)
profit: 54.32841743339861
hit recorded ratio: 0.7317073170731707 (30 out of 41)
profit: 8.397473268512897


# 2023 season prediction - 1hits model

In [168]:
feature_columns, target_column, model_file_name = model.common.features_1hits_recorded, model.common.target_1hits_recorded, model.common.model_1hits_file_name

In [179]:
regression_model = pycaret.classification.load_model(model_file_name)

Transformation Pipeline and Model Successfully Loaded


In [181]:
df_prediction_2023 = model.odds_eval.df_prediction_add_odd(df_game_matchup_2023[['game_id'] + feature_columns + [target_column]], regression_model)

In [187]:
df_prediction_odds_2023 = model.odds_eval.merge_df_prediction_over_odds(df_prediction_2023, df_odds_since_20230529[df_odds_2023.property == 'Hits'], target_column, 1.0)

In [188]:
importlib.reload(model.common)
importlib.reload(model.odds_eval)
importlib.reload(odds_data.query_bq_odds)

<module 'odds_data.query_bq_odds' from '/home/junlim/projects/mlb-props/odds_data/query_bq_odds.py'>

In [194]:
model.odds_eval.get_df_confident_prediction_odds(df_prediction_odds_2023, target_column, score_threshold = 0.75)

hit recorded ratio: 0.75 (51 out of 68)
profit: 5.215812654719599


,game_date,team_away,team_home,batting_name,batting_1hits_recorded,prediction_label,prediction_score,theo_odds,over_prob,over_odds,over_line
5053,2023-06-11,Cincinnati Reds,St. Louis Cardinals,TJ Friedl,1,1,0.7509,-301,0.714286,-250.0,0.5
1939,2023-06-24,New York Mets,Philadelphia Phillies,Nick Castellanos,1,1,0.7514,-302,0.687500,-220.0,0.5
7460,2023-06-04,Colorado Rockies,Kansas City Royals,Nick Pratto,1,1,0.7517,-303,0.661017,-195.0,0.5
2355,2023-06-18,Cincinnati Reds,Houston Astros,Kyle Tucker,1,1,0.7525,-304,0.705882,-240.0,0.5
6648,2023-06-07,Arizona Diamondbacks,Washington Nationals,Gabriel Moreno,0,1,0.7529,-305,0.733333,-275.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...
9470,2023-05-30,San Diego Padres,Miami Marlins,Luis Arraez,1,1,0.8082,-421,0.736842,-280.0,0.5
8269,2023-06-02,Colorado Rockies,Kansas City Royals,Randal Grichuk,1,1,0.8182,-450,0.722222,-260.0,0.5
6049,2023-06-08,San Francisco Giants,Colorado Rockies,Nolan Jones,1,1,0.8265,-476,0.729730,-270.0,0.5
9782,2023-05-29,Colorado Rockies,Arizona Diamondbacks,Randal Grichuk,1,1,0.8320,-495,0.672131,-205.0,0.5


In [201]:
importlib.reload(model.common)
importlib.reload(model.odds_eval)
importlib.reload(odds_data.query_bq_odds)

<module 'odds_data.query_bq_odds' from '/home/junlim/projects/mlb-props/odds_data/query_bq_odds.py'>

In [205]:
df_advantageous_prediction_odds_2023 = model.odds_eval.get_df_advantageous_prediction_odds(df_prediction_odds_2023, target_column, prediction_diff_threshold = 0.05, score_threshold = 0.70)
df_advantageous_prediction_odds_2023

hit recorded ratio: 0.8571428571428571 (6 out of 7)
profit: 1.689879297022154


,game_date,team_away,team_home,batting_name,prediction_diff,batting_1hits_recorded,prediction_label,prediction_score,theo_odds,over_prob,over_odds,over_line
2681,2023-06-18,Colorado Rockies,Atlanta Braves,Randal Grichuk,0.056583,1,1,0.7176,-254,0.661017,-195.0,0.5
474,2023-07-02,San Francisco Giants,New York Mets,Tommy Pham,0.050118,1,1,0.7560,-310,0.705882,-240.0,0.5
1939,2023-06-24,New York Mets,Philadelphia Phillies,Nick Castellanos,0.063900,1,1,0.7514,-302,0.687500,-220.0,0.5
2379,2023-06-18,Baltimore Orioles,Chicago Cubs,Christopher Morel,0.089777,1,1,0.7389,-283,0.649123,-185.0,0.5
2378,2023-06-18,Baltimore Orioles,Chicago Cubs,Austin Hays,0.083255,1,1,0.7934,-384,0.710145,-245.0,0.5
9470,2023-05-30,San Diego Padres,Miami Marlins,Luis Arraez,0.071358,1,1,0.8082,-421,0.736842,-280.0,0.5
1506,2023-06-25,Los Angeles Angels,Colorado Rockies,Nolan Jones,0.101792,0,1,0.7941,-386,0.692308,-225.0,0.5
